<a href="https://colab.research.google.com/github/aduchon/Gen_AI_Notebooks/blob/main/Aesthetic_Experiments_with_FLUX_(2024_08).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Aesthetic Experiments with FLUX (Schnell) txt2img**

https://github.com/black-forest-labs/flux

**FLUX takes only about ??? seconds per image**

Even on just a T4 (but with high CPU RAM).

The goal of this notebook is to allow the user to explore and hone in on a set of prompts that will work for a wide variety of scenes.  

If you like it, please <a href='https://ko-fi.com/L4L4SFQTN' target='_blank'><img height='36' style='border:0px;height:36px;' src='https://storage.ko-fi.com/cdn/kofi2.png?v=3' border='0' alt='Buy Me a Coffee at ko-fi.com' /></a>

In my case I have used it to find prompts that work for all the lines of a poem, so I refer to these required prompts as "lines".

**It is geared to use your Google Drive with Google Colab in a _cost effective_ manner!**  

If you want to use it locally, please adjust it yourself.

Once you have the set of lines you want to illustrate, you go through this process:

1.   A random set of prompts and parameters of many varieties are used to produce a first set of images.  The final prompts is in the form:
    1. `[line], [adjectives] [material] [art_period] [shape] by [artists] [background] [time of day] [adverbs] [image qualities]`

2.   You decide which images you like
3.   The next set of images is more likely to have those desirable parameters and prompts.  
    1. A file 'prompt_gain.pkl' in the project directory will contain all the information needed to do this.
    1. NOTE: if you like to use artists, place a simple text file, one artist per line, in the project directory and name it 'artists.txt'
        1. The project directory you name below will be created the first time you run the cell 'Path Setup' below.
    1. NOTE: you can buy a full list of over 3500 artists here: https://ko-fi.com/s/aac811be1a, or 200 photographers here: https://ko-fi.com/s/95baf9d581
4.   Repeat step 2-3 until you have enough accepted images
    1. about 200 in my experience if you need to get 20 lines all in the same style
    1. I typically will do 10 rounds and let it run over night, so with 20 lines, I have 200 images to review (the aesthetic experiment) the next night.  
    1. It takes about 40 minutes to do the review.

**NOTE:** nothing is ever deleted, files are simply moved around, e.g., into a folder called 'Deleted' in the project path, so then when you're done, you can easily delete all those files to save space.  In fact, it's critical you do not delete this folder until you are done, since it is used to determine which prompts you like more than others.

*   **Generation:** Needs GPU but can use the smallest GPU (T4)
*   **Review:** CPU only needed for review


In [ ]:
#@title **Path Setup** (always required)

from types import SimpleNamespace
from pathlib import Path
from google.colab import drive
import os

try:
    drive_path = "/content/drive"
    drive.mount(drive_path, force_remount=False)
    MYDRIVE_PATH = Path("/content/drive/MyDrive")
except:
    print("...error mounting drive or with drive path variables")

#@markdown **Path Setup**

base_dir = "AI/FLUX"
BASE_PATH = MYDRIVE_PATH / f"{base_dir}"
os.makedirs(BASE_PATH, exist_ok=True)

print(f"BASE_PATH: {BASE_PATH}")

#@markdown * under MyDrive/AI/FLUX
project_dir = "Shakespeare_FLUX_Example" #@param {type:"string"}
PROJECT_PATH = BASE_PATH / project_dir
os.makedirs(PROJECT_PATH, exist_ok=True)
print(f"PROJECT_PATH: {PROJECT_PATH}")

# where images will be put in Exploration Mode
output_image_subdir = "ForReview"
OUTPUT_IMAGE_PATH = PROJECT_PATH / output_image_subdir
os.makedirs(OUTPUT_IMAGE_PATH, exist_ok=True)

# so we only download once, store them in gdrive
huggingface_path = MYDRIVE_PATH / "AI/huggingface"
# os.makedirs(huggingface_path, exist_ok=True)

print(f"huggingface_path: {huggingface_path}")

# so models get stored here
os.environ['TRANSFORMERS_CACHE'] = str(huggingface_path / "models")
os.environ['HF_HOME'] = str(huggingface_path / "home")
os.environ['HF_DATASETS_CACHE'] = str(huggingface_path / "datasets")

# to make sure any changes get picked up immediately
%load_ext autoreload
%autoreload 2

# other paths

accepted_path = PROJECT_PATH / f"Accepted" # we want
outtake_path = PROJECT_PATH / f"Skips" # good aspects, but not this version

deleted_path = PROJECT_PATH / f"Deleted" # we don't want
keep_path = PROJECT_PATH / f"Keepers" # totally wrong but interesting

review_path = PROJECT_PATH / f"ForReview"

os.makedirs(accepted_path, exist_ok=True)
os.makedirs(deleted_path, exist_ok=True)
os.makedirs(outtake_path, exist_ok=True)
os.makedirs(keep_path, exist_ok=True)
os.makedirs(review_path, exist_ok=True)


# all sequence base directory
sequence_path = PROJECT_PATH / f"Sequences"
os.makedirs(sequence_path, exist_ok=True)

# the already accepted sequences
sequence_accepted_path = sequence_path / "SeqAccepted"
os.makedirs(sequence_accepted_path, exist_ok=True)

sequence_deleted_path = sequence_path / "SeqDeleted"
os.makedirs(sequence_deleted_path, exist_ok=True)

# sequences to be reviewed, where we will put the new ones
sequence_review_path = sequence_path / f"SeqForReview"
os.makedirs(sequence_review_path, exist_ok=True)
print(f"adding to sequences in {sequence_review_path}")


# the final set of images per set will be save here
sequence_final_path = sequence_path / "SeqFinal"
os.makedirs(sequence_final_path, exist_ok=True)
print(f"final sets will be saved to {sequence_final_path}")


# needed always
def get_settings_from_file(settings_file):
  with open(settings_file) as f:
    string = f.read()
    settings = json.loads(string, object_hook=lambda d: SimpleNamespace(**d))
    temp_dict = json.loads(json.dumps(settings, default=lambda s: vars(s)))
    cur_core_prompt_dict = temp_dict['core_prompt_dict']
    # and turn it back
    settings.core_prompt_dict = cur_core_prompt_dict
  return settings

def save_image_with_settings(image, settings, subdir=None):
  # get unique identifer from timestamp
  # timestamp = str(datetime.datetime.now().timestamp()).split(".")[0]
  timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')

  image_file = f"{timestamp}.png"
  settings_file = f"{timestamp}_settings.json"

  if subdir:
    outdir = OUTPUT_IMAGE_PATH / subdir
  else:
    outdir = OUTPUT_IMAGE_PATH

  os.makedirs(outdir, exist_ok=True)

  # print(f"saving image to {outdir/image_file}")
  image.save(outdir/image_file)

  # print(f"settings: {settings}")

  with open(outdir/settings_file, 'w')as f:
   json.dump(vars(settings), f, indent=4)

  print(f"saved {image_file} and {settings_file} TO: {outdir}")

def save_image_with_name(image, image_name, outdir):
  # for sequence expansion
  image_file = f"{image_name}.png"

  os.makedirs(outdir, exist_ok=True)

  # print(f"saving image to {outdir/image_file}")
  image.save(outdir/image_file)

  print(f"saved {image_file} TO: {outdir}")

def save_settings(settings, settings_name, outdir):

  os.makedirs(outdir, exist_ok=True)
  # print(f"settings: {settings}")
  with open(outdir/settings_name, 'w') as f:
   json.dump(vars(settings), f, indent=4)

  print(f"saved {settings_name} TO: {outdir}")




# Model Setup (for generation, GPU required)

In [ ]:
#@title installs

!pip install -q torch torchvision #
!pip install -q transformers accelerate diffusers




In [ ]:
#@title make pipeline
# https://huggingface.co/blog/lcm_lora

#@markdown NOTE: you may be asked for a HuggingFace Token (HF_TOKEN), if so, put it in your Colab "Secrets" (the key shape on the left of the screen)
import torch
from diffusers import FluxPipeline

model_id = "black-forest-labs/FLUX.1-schnell" #you can also use `black-forest-labs/FLUX.1-dev`

pipe = FluxPipeline.from_pretrained("black-forest-labs/FLUX.1-schnell", torch_dtype=torch.bfloat16)
pipe.enable_model_cpu_offload() # really need this.  save some VRAM by offloading the model to CPU. Remove this if you have enough GPU power

# pipe.enable_sequential_cpu_offload() # really slow

# needs 26Gb with or without, but a little quicker to start up and still 14s
pipe.vae.enable_slicing() #
pipe.vae.enable_tiling() # A100 needed 3min startup 14s after that



# for schnell:
pipe_steps=4


In [ ]:
#@title functions
import random
import datetime
import json
from PIL import Image
import re


def get_one_image(settings):
  # print(f"get_one_image_from_base: {settings}")

  image = pipe(
    prompt = settings.prompt,
    output_type="pil",
    num_inference_steps=4, #use a larger number if you are using [dev]
    generator=torch.Generator("cpu").manual_seed(settings.seed),
    max_sequence_length=256,
    width=settings.width,
    height=settings.height,
    guidance_scale=0.0,

  ).images[0]

  return image


def get_images_direct(settings, key, save_dir):
  # just get images for the prompts directly

  print(f"get_images_direct: {settings}")

  images_created = 0

  for prompt_num, line in enumerate(settings.prompts):
    line_words = re.sub(f'[^\w\s]', '',line)
    line_text = "_".join(line_words.split()[:4])
    outfile = save_dir / f"{key}_{prompt_num:04}_{line_text}.png"

    if outfile.exists():
      print(f"EXISTS: {outfile}")
      continue

    print(f"getting image ({prompt_num}): {line}")

    settings.prompt = line + ", " + settings.tail_prompt
    image = get_one_image(settings)

    print(f"Saving image to: {outfile}")
    image.save(outfile)
    images_created += 1

  return images_created

def image_grid(imgs):
  if len(imgs) >= 4:
    cols = 4
    rows = 1 + len(imgs) // cols
  else:
    rows = 1
    cols = len(imgs)

  w, h = imgs[0].size
  if w>512 or h>512:
    w = w//2
    h = h//2
  grid = Image.new('RGB', size=(cols*w, rows*h))
  grid_w, grid_h = grid.size

  for i, img in enumerate(imgs):
    grid.paste(img.resize((w,h)), box=(i%cols*w, i//cols*h))

  return grid

def get_token_count(prompt):
  # text_inputs = base.tokenizer(
  #       prompt,
  #       padding="max_length",
  #       max_length=base.tokenizer.model_max_length,
  #       truncation=True,
  #       return_tensors="pt",
  #   )
  # text_input_ids = text_inputs.input_ids
  # untruncated_ids = base.tokenizer(prompt, padding="max_length", return_tensors="pt").input_ids
  untruncated_ids = pipe.tokenizer(prompt, padding=False, return_tensors="pt").input_ids

  # print(text_input_ids[0])
  # print(untruncated_ids[0])

  # if not torch.equal(text_input_ids, untruncated_ids):
  #   removed_text = base.tokenizer.batch_decode(untruncated_ids[:, base.tokenizer.model_max_length - 1 : -1])
  #   print(
  #       "The following part of your input was truncated because CLIP can only handle sequences up to"
  #       f" {base.tokenizer.model_max_length} tokens: {removed_text}"
  #   )
  return len(untruncated_ids[0])




# Line Prompts (always required)

In [ ]:
#@title basic lines (open cell to change)


# Update this with your lines.

# Shakespeare's Sonnet 154
# two lines only for testing
lines = [
  "The little love-god lying once asleep",
  "Laid by his side his heart-inflaming brand,",
  # "Whilst many nymphs that vow'd chaste life to keep",
  # "Came tripping by; but in her maiden hand",
  # "The fairest votary took up that fire",
  # "Which many legions of true hearts had warm'd;",
  # "And so the general of hot desire",
  # "What sleeping by a virgin hand disarm'd.",
  # "This brand she quenched in a cool well by,",
  # "Which from Love's fire took heat perpetual,",
  # "Growing a bath and healthful remedy",
  # "For men diseased; but I, my mistress' thrall,",
  # "Came there for cure, and this by that I prove,",
  # "Love's fire heats water, water cools not love.",

]


print(f"lines {len(lines)}: {lines}")


# Complete prompt lists (always required)

1. Open the cells to add/change

1. Feel completely free to add, or comment out (put a # in front of) any of these to focus the generation of images.
1. just have "\_" if you don't want it used, or add "\_" if want the option for this type of prompt to be blank
1. If you want to include artists, make sure there is a file 'artists.txt' in the PROJECT_PATH

In [ ]:

# keep track of the prompt types
all_prompt_types = set()


In [ ]:
#@title art_period

all_prompt_types.add('art_period')

art_period = [

"Roman",
"Greek",
"Medieval",
"Gothic",
"Renaissance",
"Cretan School",
"Mannerism",
"Baroque",
"Rococo",
"Neoclassicism",
"Romanticism",
"Academic art",
"Realism",
"Macchiaioli",
"PreRaphaelite",
"Naturalism",
"Art Nouveau",
"Art Deco",
"Impressionism",
"Post-Impressionism",
"Neo-Impressionism",
"Neo-Expressionism",
"Fauvism",
"Expressionism",
"Tonalism",
"Cubism",
"Surrealism",
"Futurism",
"Abstract Expressionism",
"Avantgarde",
"Bauhaus",
"Op Art",
"Pop Art",
"Constructivism",
"Suprematism",
"New Objectivity",
"Symbolism",
"Vorticism",
"Biomorphism",
"De Stijl",
"Socialist",
"Dadaism",
"Kinetic Art",
"Futurism",
"Harlem Renaissance",
"Arte Povera",
"Zero Group",
"Minimalism",
"Conceptual Art",
"Contemporary Art",
"Lowbrow",
"Modernism",
"Deconstuctionism",
"Post-Modern",
"Maximalist",
"Massurealism",
"Stuckism",
"Remoderism",
"Excessivism",
"art", #
"modern art",
"digital art",

"_",

 "classic painting",
 "historical painting",
"jazz age",
 "atomic era",

"abstract minimalism",



]

print(f"art_period {len(art_period)}: {art_period}")


In [ ]:
#@title materials
# use https://huggingface.co/spaces/pharma/CLIP-Interrogator

all_prompt_types.add('material')

material = [
"acrylic painting",
"airbrush painting",
"aquatint",
"ballpoint pen art",
"banhua",
"black velvet",
"bokashi",
"brayer painting",
"carborundum print",
"cartoon",
"casein painting",
"catchpenny print",
"cel animation",
"chalk",
"character animation",
"charcoal drawing",
"chibi art",
"children’s story book",
"chine-collé",
"chromolithography",
"chromoxylography",
"cliché verre",
"collage",
"collagraphy",
"colored pencil",
"comic",
"cordel literature",
"crayon",
"decoupage",
"disney",
"doodle",
"drypoint",
"e-hon",
"encaustic",
"engraving",
"epinal print",
"etching",
"fan art",
"finger painting",
"frescoe painting",
"frescography",
"fudezaishiki",
"geomontography",
"giclée",
"glitter painting",
"gond painting",
"gouache painting",
"graphite drawing",
"grease pencil",
"grisaille",
"heures de charles d'angoulême",
"illuminated manuscript",
"illustrated book",
"illustrated childrens book",
"illustration for children",
"illustration",
"impasto",
"ink-wash",
"japonisme",
"kappazuri",
"kuchi-e",
"limning",
"linocut",
"lithography",
"looney tunes",
"madhubani painting",
"metalpoint",
"mixed media",
"mosaic",
"mural",
"nib painting",
"oil painting",
"origami",
"pabalat",
             "painting",
"palette knife",
"papel picado",
"pastel",
"pen drawing",
"pencil drawing",
"permanent marker",
"phad painting",
"picture book",
"pixar",
"pop-up book",
"puppet film",
"relief printing",
"saturday morning cartoon",
"screen print",
"screengrab",
"sfumato",
"silverpoint",
"sketch",
"spray paint",
"stele rubbing",
"stencil art",
"stereochromy",
"stereoscopic image",
"stick figure",
"stop motion",
"tempera",
"trois crayons",
"ukiyo-e",
"vitreography",
"vytynanky",
"warli painting",
"watercolor",
"whiteboard art",
"wimmelbilderbuch",
"wycinanki",

    "_",
]
print(f"material ({len(material)}): {material}")

In [ ]:
#@title adjectives

# https://github.com/WASasquatch/noodle-soup-prompts/blob/main/nsp_pantry.json

# can be only single words
# positive for childrens books
all_prompt_types.add('adjectives')

adjectives = [
"abandoned",
"abhorrent",
"absurdist",
"acclaimed",
"accomplished",
"adroit",
"aesthetic",
"alien",
"alluring",
"aloof",
"amazing ",
"amber",
"amused",
"angry",
"anxious",
"apalling",
"apocalyptic",
"appealing",
"artistic",
"arty",
"astonishing",
"atmospheric",
"attractive",
"authentic",
"avant-garde",
"award-winning",
"awe-inspiring",
"awe-struck",
"awry",
"baffled",
"balanced",
"basic",
"beauteous",
"beautiful",
"bewildered",
"bleak",
"bold",
"boundless",
"bright",
"brilliant",

"calming",
"camp",
"candid",
"catastrophic",
"ceramic",
"chaotic",
"characteristic",
"charming",
"classic",
"clever",
"collectable",
"colorful",
"colossal",
"comical",
"complex",
"confident",
"contemplative",
"contemporary",
"content",

"crafty",
"creative",
"crippling",
"cultured",
"curious",
"cursed",
"cute",
"daring",
"dazzling",
"decaying",
"decorative",
"delicate",
"desolate",
"desperate",
"detailed",
"determined",
"devastated",
"devastating",
"devoured",
"disappointed",
"disastrous",
"disciplined",
"disgusted",
"disheartening",
"dismal",
"distinctive",
"disturbing",
"divine",
"doomed",
"dramatic",
"dreamlike",
"dreamy",
"dreary",
"dynamic",
"eclectic",
"eerie",
"elated",
"elegant",
"elevated",
"emotional",
"enchanted",
"enchanting",
"energetic",
"engaging",
"engrossing",
"enigmatic",
"enthusiastic",
"enticing",
"envious",
"esthetical",
"ethereal",
"evocative",
"exceptional",
"excited",
"expressive",
"exquisite",
"extreme",
"eye-catching",
"fanciful",
"fascinating",
"fashionable",
"fearful",
"figural",
"figurative",
"flawless",
"fluid",
"folk",
"folksy",
"forlorn",
"formal",
"freelance",
"fresh",
"frightening",
"frightful",
"frustrated",
"fun",
"funny",
"gaudy",
"genius",
"ghastly",
"gifted",
"gigantic",
"glamorous",
"gloomy",

"gorgeous",
"gory",
"graceful",
"grand",
"grandiose",
"grim",
"gruesome",
"guilty",
"handsome",
"happy",
"harmonious",
"harrowing",
"haunting",
"heart-wrenching",
"honest",
"hopeful",
"hopeless",
"horrendous",
"horrifying",
"hued",
"humorous",
"hyper",
"hyper-creative",
"imaginative",
"immense",
"impassioned",
"impatient",
"impeccable",
"impossible",
"infused",
"inspirational",
"inspired",
"inspiring",
"instinctive",
"intellectual",
"intense",
"interesting",
"interpretive",
"intuitive",
"inventive",
"joyful",
"knockout",
"labyrinthine",

"layered",
"light",
"liquid",
"literary",
"luminous",
"lyrical",
"macabre",
"magical",
"magisterial",

"massive",
"melancholic",
"memorable",
"miraculous",
"miserable",
"monstrous",
"monumental",
"mournful",
"moving",
"mundane",
"musical",
"mysterious",
"mystical",
"narrative",
"naturalistic",
"nauseating",
"nervous",
"nonchalant",
"nubile",

"oppressive",


"organic",
"original",
"pained",
"paradoxical",
"passionate",
"patina",
"peaceful",

"pensive",
"perfect",
"personable",
"petrifying",
"phenomenal",
"philosophical",
"picturesque",
"playful",
"pleasant",
"poetic",
"pretty",
"pure",
"questionable",
"radiant",
"ravishing",
"regretful",
"relieved",
"religious",
"remarkable",
"rhythmical",
"rich",
"romantic",

"ruined",
"sad",

"satire",
"saturated",
"sensual",
"sensuous",
"serene",
"shocking",
"showstopping",
"shy",
"simple",
"smart",
"soft",
"sorrowful",
"spacey",
"sparse",
"spiritual",
"statuesque",
"stimulating",
"stirring",
"studied",
"stunning",
"stylish",
"stylized",
"sublime",
"substantive",
"superb",
"supernatural",
"supple",
"surprised",
"surreal",
"symbolic",
"tasteful",
"telegenic",
"traditional",
"tragic",
"tranquil",
"trendy",
"turquoise",
"unconventional",
"unexpected",
"unimaginable",
"unique",
"universal",
"unpredictable",
"unpretentious",
"vibrant",
"visionary",
"vivid ",
"weird",
"whimsical",
"wretched",
"zingy",

               "and more and more and more",
               "etc.",

# # Colors
# "aqua blue",
# "aquamarine",
# "beige",
# "blue",
# "bronze colored",
# "burgundy red",
# "champagne yellow",
# "chartreuse",
# "copper colored",
# "coral orange",
# "crimson",
# "cyan",
# "emerald green",
# "fuchsia",
# "gold colored",
# "green",
# "indigo",
# "ivory white",
# "lavender purple",
# "lime green",
# "magenta",
# "maroon",
# "mint green",
# "mauve",
# "mustard yellow",
# "navy blue",
# "olive green",
# "orange",
              #  "orchid",
# "peach red",
# "pearl white",
# "periwinkle",
# "pink",
# "plum red",
# "purple",
# "red",
# "rose red",
# "ruby red",
# "salmon orange",
# "sapphire red",
# "scarlet red",
              #  "sepia",

# "silver colored",
# "tan brown",
              #  "teal",

# "violet",
# "vermillion",
# "yellow",

"_",
]


print(f"adjectives ({len(adjectives)}): {adjectives}")


In [ ]:
#@title background
all_prompt_types.add('background')

background = [

  "_",

  # "forest",
  # "city",
  # "beach",
  # "mountains",
# "huddled masses of the poor",


]

print(f"background ({len(background)}): {background}")



In [ ]:
#@title time of day
all_prompt_types.add('time_of_day')

time_of_day = [
"_",
# "sunrise",
#                 "sunset",
#                 "golden hour",

]

print(f"time_of_day ({len(time_of_day)}): {time_of_day}")



In [ ]:
#@title image qualities
all_prompt_types.add('image_qualities')

image_qualities = [
"2019",
"21:9",
                    "32k UHD",
"35mm",
"4k",
"8k",
"artstation",
"atmospheric",
"award-winning",
"cinematic lighting",
"deep depth of field",
                    "detailed",
"deviant art",
"dramatic lighting",
"dslr camera",
"exquisite details",
"exquisite textures",
"extreme long shot",
"extreme wide shot",
"extremely detailed",
"f/22",
"fantastic backlight",
"full shot",
"hdr",
"highly detailed",
"high resolution",
"high definition",
"hyper detailed",
"hyper realistic",
"hyperrealism",
"image in center",
"instagram contest winner",
"intricate details",
"kodak pro gold",
"large format",
"ldsr camera",
"long shot",
"maximum texture",
"national geographic",
"medium format",
"perfect lighting",
"perfect composition",
"photo courtesy museum of art",
"photography",
"rim lighting",
"rule of thirds",
"sharp features",
"sharp focus",
"sharp",
"anamorphic lenses",
"shutterstock contest winner",
"smooth",
"soft lighting",
"studio light",
"theatrical",
"trending on artstation",
"ultra detailed",
"ultra photoreal",
"ultra realistic",
"unreal engine",
"very crisp",
"very detailed",
"volumetric",
"wide angle lens",
"wide shot",
"widescreen shot",

#lenses:
# "Sigma 50mm T1.5 FF High-Speed Prime",
"OM-D E-M5 Mark III",
# "M.Zuiko Digital ED 12–40mm F2.8 PRO",
"1/50sec",
"ISO64",
"OM system 12–40mm PRO",
# "Samyang/Rokinon Xeen 50mm T1.5",
"Sigma 40mm f/1.4 DG HSM",
"landscape",

# new lenses
"Nikon Z9 200 mm lens",

"clean detailed faces",
"intracate clothing",
"analogous colors",
"glowing shadows",
"beautiful gradient",
"depth of field",
"clean image",


 # film types
 "Kodak Ektachrome E100",
 "Fujifilm Pro 400H",
 "Agfa Vista 400",
 "Kodak Gold 100",
 "Kodak Gold 200",
 "Kodak Portra 400",
 "Kodak Ektar 100",
 "Fujichrome Velvia 50",
 "CineStill 800T",
 "Fujifilm Superia 400",
"AgfaColor Neu",
  "Anscochrome",
  "Fujifilm Velvia",
  "Agfa CT Precisa",
                    "Lomography Color Negative 800",
"Fujifilm Provia film",
"ISO 400",
"Kodak film",

  # TODO: https://aesthetics.fandom.com/wiki/List_of_Aesthetics

  "associated press photo",
  "AP photo",
  "UHD",


  # from clip interegator

  "awesome composition",
  "trending on cgstation",
  "high detailed official artwork",
  "9k",
  "official artwork",

  "mesmerizing shot",
  # "",
  # "",
# "shot in a realistic and cinematic style with RED Weapon Monstro 8K VV",
"shot on hasselblad",
"shot with Canon EF on Kodak Portra 800 film",
"shot with a Hasselblad camera",
                    "the image boasts incredible realism",
# "tokina at-x 11-16mm f/2.8 pro dx ii",
                    "85mm lens",
# "Arri Alexa Mini LF with a Cooke S7/i 35mm T2.0 lens",
"Contax T3 SLR",

"Leica 85mm lens",
"Leica Summilux-C Prime Lenses.",
"Matte",
# "Sigma 150-600mm f/5-6.3 DG OS HSM Sports lens at 600mm",
"Sony a7s III",
                    "aesthetic photostrip from a photobooth",
"cartridge-loaded",
"chemical-processed",
                    "contrasting color",
"contrasting softness",
                    "digital art smooth",
"dreamlike aura",
"natural lighting",
                    "nikon d850",
"non-digital",
"pentax 645n",
                    "stunning color grading",
"light-sensitive",
"point-and-shoot",
"perfect color graded",
"beautifully color-coded",
"cinematic look",
"cinematic shot",
                    "contest winner",
"fixed-aperture",
"fixed-focus",
"flash-enabled",
      "global illumination",
"glossy",
"grainy",
"handheld",
                    "cartridge-loaded",
"chemical-processed",
                    "contrasting color",
"contrasting softness",
                    "cinematic dark atmosphere",
                    "gloomy and foggy atmosphere",
     "night cinematic lighting",
       "supersharp",
                          "sharp edges",
                    "jagged edges",



]

print(f"image_qualities ({len(image_qualities)}): {image_qualities}")


In [ ]:
#@title adverbs

all_prompt_types.add('adverbs')

adverbs = [

"admirably",
"aesthetic joys of dance ",
"affectionately ",
"amazing contrast of tenderness and cruelty",
"anatomical",
"anatomically",
"badly",
"benevolently ",
"boastfully",
"boldly",
"bravely",
"breathtakingly",
"brightly",
"brilliantly",
"busily",
"calmly",
"carefully",
"caringly",
"celebrated works",
"cheerfully",
"choreographed",
"classic model",
# "clearly",
"complex and ever-changing emotions",
"courageously",
"cruelly",
"daily",
"dramatically",
"dripping ",
"easily",
"ecstatically",
"elegantly",
"emotion-packed",
"enormously",
"enthusiastically",
"exactly",
"expressive faces",
"exultation",
"faithfully",
"fiercely",
"finesse",
"fondly",
"foolishly",
"fortunately",
"gently",
"gladly",
"gracefully",
"graciousness",
"greedily",
"happily",
"honestly",
"innocently",
"intricately",
"inventive",
"joyfully",
"joyously",
"kindly",
"laughingly",
"lyrically",
"manifest ability",
"merrily",
"neatly",
"physical interpretation",

"reimagined",

"shimmering ",
"smooth skin",

"strong emotion",
"strong dynamic pose",
"swiftly",
"tenderly",
"warmly",
"wheeling",
 "tense and dynamic atmosphere",
"dynamic image immerses viewers",
"exaggerated facial features",
"juxtaposition of ethereal beauty and unsettling transformation",
"reminiscent of a dreamlike",
"emotive body language",


 "_",


]
print(f"adverbs ({len(adverbs)}): {adverbs}")



In [ ]:
#@title shape
all_prompt_types.add('shape')

shape = [
"_",
"photograph",
"painting",


]
print(f"shape ({len(shape)}): {shape}")


In [ ]:
#@title artists from files
all_prompt_types.add('artists')

from pathlib import Path
!pip install unidecode
from unidecode import unidecode

artists = set()

artists_file = PROJECT_PATH / "artists.txt"

print(f"getting artists from : {artists_file}")

try:
  with open(artists_file, 'r', encoding='UTF-8') as file:
    for line in file:
      # print(line.strip())
      artists.add(unidecode(line.strip()))

  # turn set into a list
  artists = list(artists)
  print(f"Found {len(artists)} in {artists_file}")
except Exception as e:
  print(f"NO file: {artists_file}")
  artists = ["_"]

print(f"Found {len(artists)}")


In [ ]:
#@title numeric options

all_prompt_types.add('steps')
all_prompt_types.add('guidance_scale')

# what was chosen at the top
steps = [pipe_steps] #

# they suggest just 0.0, and no negative prompt
guidance_scale = [0.0] #



# Core Prompt Functions (always required). Also, rerun after getting final prompt votes (Step 1c)

In [ ]:
#@title core_prompt function

import random
import re
import pickle
from collections import OrderedDict
import numpy as np
import pprint


# global
cur_core_prompt_dict = {}

test_weights = False

# load the prompt gain
gain_path = PROJECT_PATH / "prompt_gain.pkl"
print(f"using gain_path: {gain_path}")


def update_prompt_type(prompt_type):
  # to convert from sd2.1 to SDXL styles

  if prompt_type == 'art_periods':
    prompt_type_out = 'art_period'
  elif prompt_type == 'materials':
    prompt_type_out = 'material'
  elif prompt_type == 'scale':
    prompt_type_out = 'guidance_scale'
  else:
    prompt_type_out = prompt_type

  return prompt_type_out


def set_prompt_probs(prompt_type):

  # converting from old dict keys to new:
  new_prompt_type = update_prompt_type(prompt_type)

  if new_prompt_type not in all_prompt_types:
    print(f"prompt_type deprecated: {prompt_type}")
    return None, None

  # create an ordered dict
  ordered_option2weight = OrderedDict(prompt_gain[prompt_type])


  print(f"{prompt_type}: current weights ({len(ordered_option2weight)}): {ordered_option2weight}")

  # will only include items not currently excluded
  cur_items = [(update_prompt_type(i), ordered_option2weight[i]) for i in ordered_option2weight.keys() if i in eval(new_prompt_type)]

  # add in any new items with a wt of 1
  cur_keys = [i for i,w in cur_items]
  for item in eval(new_prompt_type):
    if item not in cur_keys:
      # print(f"adding to {k}: {item}")
      cur_items.append((item, 1))


  total_cnt = sum([wt for i, wt in cur_items])
  options = [i for i, wt in cur_items]
  probs = [wt / total_cnt for i, wt in cur_items]
  return options, probs

# TODO: if check for any new prompt types give them even probs

can_use_weights = False
# for each type, set the list of options and probs once
prompt_type2options = {}

if gain_path.exists():
  # this path won't exist until the first voting which will make sure every
  # known prompt gets a value of 1
  can_use_weights = True

  print(f"Loading gain from {gain_path}")
  with open(gain_path, 'rb') as handle:
      prompt_gain = pickle.load(handle)

  print(f"prompt_gain: keys={prompt_gain.keys()}")

  # set the dictionary
  for k in prompt_gain.keys():

    options, probs = set_prompt_probs(k)
    if not options:
      continue


    k = update_prompt_type(k)
    prompt_type2options[k] = {}
    prompt_type2options[k]['options'] = options
    prompt_type2options[k]['probs'] = probs

  # get any new prompts
  for k in all_prompt_types:
    if k not in prompt_type2options:
      print(f"new prompt_type: {k}")
      total_cnt = len(eval(k)) # the number of items
      options = eval(k)
      probs = [1 / total_cnt] * total_cnt
      prompt_type2options[k] = {}
      prompt_type2options[k]['options'] = options
      prompt_type2options[k]['probs'] = probs


  # print(f"final:")
  # pprint.pprint(prompt_type2options)


def get_sample(prompt_type, option_cnt):
  if not prompt_type in prompt_type2options:
    return ["_"]
  if not prompt_type2options[prompt_type]['options']:
    return ["_"]

  options_to_use = np.random.choice(prompt_type2options[prompt_type]['options'],
                    replace=False,
                    p=prompt_type2options[prompt_type]['probs'],
                    size=option_cnt, )

  # print(f"{prompt_type}: options_to_use ({len(options_to_use)}): {options_to_use}")
  return options_to_use.tolist()


def get_core_prompt():
  prompt_type2cnt = {}
  for prompt_type in all_prompt_types:
    # most types get just 1
    prompt_type2cnt[prompt_type] = 1

  prompt_type2cnt['artists'] = random.randint(0, min(6, len(artists)))
  prompt_type2cnt['adjectives'] = random.randint(1, min(3, len(adjectives)))
  prompt_type2cnt['adverbs'] = random.randint(2, min(4, len(adverbs)))
  prompt_type2cnt['image_qualities'] = random.randint(2, min(4, len(image_qualities)))

  cur_dict = {}

  # use the weight values
  if can_use_weights:
    for prompt_type in all_prompt_types:
      cur_dict[prompt_type] = get_sample(prompt_type, prompt_type2cnt[prompt_type])

  else:
    for prompt_type in all_prompt_types:
      cur_dict[prompt_type] = random.sample(eval(prompt_type), prompt_type2cnt[prompt_type])

  print(f"cur_core_prompt: {cur_dict}")
  return cur_dict

def get_prompt_for_line(line_num):
  #
  #  [line], [color] [art_period] [material] [shape] by [artists]

  # need to check token count of entire prompt, must be less than 77

  token_count = 100
  while token_count > 77:
    out_prompt = lines[line_num]

    # print(f"line prompt: {out_prompt}")

    # everything is a list, so have to concatenate each type
    out_prompt = out_prompt \
                  + ", " \
                  + ", ".join(cur_core_prompt_dict['adjectives'])  \
                  + " " \
                  + " " + ", ".join(cur_core_prompt_dict['material'])  \
                  + " " \
                  + " " + ", ".join(cur_core_prompt_dict['art_period'])  \
                  + " " \
                  + " " + ", ".join(cur_core_prompt_dict['shape'])  \
                  + " " \
                  + " by the artists " + " and ".join(cur_core_prompt_dict['artists']) \
                  + " " \
                  + ", " + ", ".join(cur_core_prompt_dict['background'])  \
                  + " " \
                  + ", " + ", ".join(cur_core_prompt_dict['time_of_day'])  \
                  + " " \
                  + ", " + ", ".join(cur_core_prompt_dict['adverbs']) \
                  + " " \
                  + ", " + ", ".join(cur_core_prompt_dict['image_qualities'])

    out_prompt = re.sub("_", "", out_prompt)
    out_prompt = re.sub("\s+", " ", out_prompt)
    out_prompt = re.sub("(\,\s){2,}", ", ", out_prompt)

    token_count = get_token_count(out_prompt)
    print(f"prompt token_count ({token_count}): {out_prompt}")

    if token_count > 77:
      # remove from the last item in image_qualities
      if len(cur_core_prompt_dict['image_qualities']) > 0:
        # get rid of the last item
        removed = cur_core_prompt_dict['image_qualities'].pop()
        print(f"{token_count}: removed from image_qualities: {removed}")
      elif len(cur_core_prompt_dict['adverbs']) > 0:
        # do these second if still too long
        # get rid of the last item
        removed = cur_core_prompt_dict['adverbs'].pop()
        print(f"{token_count}: removed from adverbs: {removed}")
      elif len(cur_core_prompt_dict['artists']) > 0:
        # do these third if still too long
        # get rid of the last item
        removed = cur_core_prompt_dict['artists'].pop()
        print(f"{token_count}: removed from artists: {removed}")
      else:
        raise ValueError(f"token_count {token_count} is still too long")

  return out_prompt




In [ ]:
# testing

try:
    cur_core_prompt_dict = get_core_prompt()
    print(f"cur_core_prompt_dict: {cur_core_prompt_dict}")
    prompt = get_prompt_for_line(0)
    print(f"prompt: {prompt}")

    # get weighted options
    if can_use_weights:
      print("\nfrom weights:")
      print(f"scale: {get_sample('guidance_scale', 1)[0]}")
      print(f"steps: {get_sample('steps', 1)[0]}")

  # TODO: get the new scheduler name
except Exception as e:
  print(e)


#Sample Exploration (GPU required)

In [ ]:
#@title Sample Exploration

from IPython import display

#@title Sample Exploration Meta-Run (to narrow down core_prompt)

# get the available core prompts
# get a random set of the core prompts
# combine and run the meta as many times as given
# go through each line

import random
import time
import gc
import glob
from pathlib import Path
import re

attempts = 20  #@param
kill_when_done = False #@param{type:"boolean"}

#@markdown much faster if you don't show images
show_images = False #@param{type:"boolean"}

#@markdown if greater than zero will only do the first max_lines
max_lines = 0 #@param{type:"integer"}

#@markdown if greater than zero will start at min_line
min_lines = 0 #@param{type:"integer"}

#@markdown https://stablediffusionxl.com/sdxl-resolutions-and-aspect-ratios/

width = 1344 #@param{type:"integer"}
height = 768 #@param{type:"integer"}

#@markdown Can take a few minutes for the first image


def get_line_name(line_number):
  # get a short version of the line
  out_name = lines[line_number]
  # Remove all non-word characters (everything except numbers and letters)
  out_name = re.sub(r"[^\w\s]", '', out_name)
  # Replace all runs of whitespace with a single dash
  out_name = re.sub(r"\s+", '-', out_name)
  # get the first 3 words as the outname
  out_name = "-".join(out_name.split("-")[:3])
  return out_name


for a in range(attempts):
  display.clear_output(wait=True)

  for line_number in range(len(lines)):
    if max_lines > 0 and line_number >= max_lines:
      continue
    if min_lines > 0 and line_number < min_lines:
      continue

    image_settings = SimpleNamespace()
    # get a random instance of the core prompt dict, global
    cur_core_prompt_dict = get_core_prompt()

    # these are likely fixed
    image_settings.width = width
    image_settings.height = height
    image_settings.line_number = line_number

    # where will put the out image
    image_settings.line_name = get_line_name(line_number)
    subdir = f"line_{line_number:02}_{image_settings.line_name}"
    print(f"round: {a+1}; batch_name: {subdir}")

    image_settings.prompt = get_prompt_for_line(line_number)
    # getting the prompt for the specific line, may change image_qualities in cur_core_prompt_dict
    # so set this after getting the prompt
    image_settings.core_prompt_dict = cur_core_prompt_dict

    # to show all the settings:
    # pprint.pprint(image_settings.core_prompt_dict)

    # print(f"line {line_number+1:2}/{len(lines):2}: attempt {a+1:2} /{attempts:2}: {image_settings.prompt}")
    print(f"\tprompt  : {image_settings.prompt}")

    # number values
    image_settings.guidance_scale = cur_core_prompt_dict['guidance_scale'][0]
    image_settings.steps = cur_core_prompt_dict['steps'][0]
    print(f"scale: {image_settings.guidance_scale}; steps:{image_settings.steps}")

    image_settings.seed = random.randint(1,2147483647)
    output_image = get_one_image(image_settings)

    save_image_with_settings(output_image, image_settings, subdir=subdir)

    if show_images:
      display.display(output_image)

    # clean up unused memory
    gc.collect()
    torch.cuda.empty_cache()


if kill_when_done:
  # kill switch when done
  from google.colab import runtime
  runtime.unassign()
  print("DONE!")




# Prompt Voting (no GPU required)

In [ ]:
#@title Prompt Voting

# go through the directories with a prefix
# show the image to the user
# (a)dd or (s)kip; add=good style, skip=bad style
# if an add, then read the json for that image, pull out the prompts
# add a counter dictionary for each prompt, at the end or (q)uit, show the counts
from pathlib import Path
import os, re
import glob
from google.colab.patches import cv2_imshow
import cv2
import time
import json
from collections import defaultdict
from tqdm import tqdm

from PIL import Image

def remove_empty_dir(dir):
  #remove directory if empty
  if len(os.listdir(dir)) == 0:
    try:
      print(f"trying to remove empty dir: {dir}")
      os.rmdir(dir)
      print(f"\tremoved empty dir: {dir}")
    except Exception as e:
      print(e)
      pass

def move_files_to_path(files, path):

  print(f"moving {len(files)} to {path}")
  # move the skipped files
  for f in files:
    file_path = Path(f)
    # and the settings file
    core_file_str = str(file_path.stem)
    timestamp = core_file_str.split("_")[0]
    json_file = file_path.parent / f"{timestamp}_settings.json"
    # move them
    try:
      file_path.rename(path / file_path.name)
      json_file.rename(path / json_file.name)
      #print(f"move {f} TO {outtake_path / file_path.name}")
    except Exception as e:
      print(f"ERROR moving: {f}")
      print(e)

    remove_empty_dir(file_path.parent)


input_files = [f
               for f in map(Path, sorted(glob.glob(f"{review_path}/*/*")))
               if f.is_file() and f.suffix in [".jpg", ".png"]
               ]

total_file_cnt = len(input_files)

print(f"input_files {total_file_cnt}: {input_files}")

accepted_files = []
delete_files = []
skip_files = []
keep_files = []

print("Loading images")
file2image = {}
for i in tqdm(range(total_file_cnt)):
  file2image[str(input_files[i])] = cv2.imread(str(input_files[i]))


quit = False
for i in range(total_file_cnt):
  if quit:
    break

  if file2image[str(input_files[i])].shape[0] > 1000 or file2image[str(input_files[i])].shape[1] > 1000:
    small = cv2.resize(file2image[str(input_files[i])], (0,0), fx=0.5, fy=0.5)
    cv2_imshow(small)
  else:
    cv2_imshow(str(file2image[input_files[i]]))

  # get the full line:
  file_name = input_files[i]
  line_num = int(file_name.parent.stem.split("_")[1])
  print(lines[line_num])

  accepted = False
  time.sleep(1)

  # have to have this sleep for the input box to show up
  while not accepted:
    try:
      print(f"img : {input_files[i]}")
      someInput = input("Accept: ")
      if someInput == "a":
        print(f"\tAccepted: {len(accepted_files):4} / {len(skip_files):4} / {len(delete_files):4} / {len(keep_files):4} / {total_file_cnt:4} ")
        accepted = True
        accepted_files.append(input_files[i])
      elif someInput == "d":
        print(f"\tDeleting: {len(accepted_files):4} / {len(skip_files):4} / {len(delete_files):4} / {len(keep_files):4} / {total_file_cnt:4} ")
        delete_files.append(input_files[i])
        accepted = True
      elif someInput == "s":
        print(f"\tSkipping: {len(accepted_files):4} / {len(skip_files):4} / {len(delete_files):4} / {len(keep_files):4} / {total_file_cnt:4} ")
        skip_files.append(input_files[i])
        accepted = True
      elif someInput == "k":
        print(f"\tKeeping: {len(accepted_files):4} / {len(skip_files):4} / {len(delete_files):4} / {len(keep_files):4} / {total_file_cnt:4} ")
        keep_files.append(input_files[i])
        accepted = True
      elif someInput == "q":
        print("Quitting")
        accepted = True
        quit = True
      elif someInput == "u":
        print("Upscaling")
        accepted = False
        quit = False
        cv2_imshow(file2image[input_files[i]])
    except Exception:
      pass

print(f"accepted_files {len(accepted_files)}")
print(f"delete_files {len(delete_files)}")
print(f"skip_files {len(skip_files)}")
print(f"keep_files {len(keep_files)}")

move_files_to_path(accepted_files, accepted_path)
move_files_to_path(delete_files, deleted_path)
move_files_to_path(skip_files, outtake_path)
move_files_to_path(keep_files, keep_path)

print(os.listdir(PROJECT_PATH))




In [ ]:
#@title get the final prompt votes


from pathlib import Path
import os
import glob
import json
from collections import defaultdict
import time

!pip install unidecode

from unidecode import unidecode

min_count_to_show = 4 #@param

accepted_path = PROJECT_PATH / f"Accepted"
deleted_path = PROJECT_PATH / f"Deleted"
skipped_path = PROJECT_PATH / f"Skips"
kept_path = PROJECT_PATH / f"Keepers"


prompt_gain = {}

def prompt_counter_from_path(path, prompt_type):
  print(f"getting prompt counts for ({prompt_type}) from {path}")
  files = [str(f)
               for f in map(Path, sorted(glob.glob(f"{path}/*")))
               if f.is_file() and f.suffix in [".jpg", ".png"]
               ]
  print(f"got {len(files)} from {path}")

  prompt_counter = defaultdict(int)
  prompt_rank = defaultdict(int)

  for f in files:
    # get the json file of the image
    file_path = Path(f)
    core_file_str = str(file_path.stem)
    timestamp = core_file_str.split("_")[0]
    json_file = file_path.parent / f"{timestamp}_settings.json"
    #print(f"json: {json_file}")
    try:
      with open(json_file) as json_file_open:
        params = json.load(json_file_open)
        # everything is a list now
        for k in params['core_prompt_dict'].keys():
          if k != prompt_type:
            continue
          # print(f"k: {k}")
          for r, p in enumerate(params['core_prompt_dict'][k]):
            if type(p) == str:
              p = unidecode(p.strip())
            else:
              pass
            # print(f"p: {p} at {r}")
            if p:
              prompt_counter[p] += 1.0
              prompt_rank[p] += float(r)
    except Exception as e:
          print(e)
          time.sleep(10)

  return prompt_counter, prompt_rank, prompt_counter

def param_counter_from_path(path, param):
  # get counts of the param
  print(f"getting sampler counts for {param} from {path}")
  files = [str(f)
               for f in map(Path, sorted(glob.glob(f"{path}/*")))
               if f.is_file() and f.suffix in [".jpg", ".png"]
               ]
  print(f"param_counter_from_path: got {len(files)} from {path}")

  sampler_counter = defaultdict(int)

  for f in files:
    # get the json file of the image
    file_path = Path(f)
    core_file_str = str(file_path.stem)
    timestamp = core_file_str.split("_")[0]
    json_file = file_path.parent / f"{timestamp}_settings.json"
    #print(f"json: {json_file}")
    try:
      with open(json_file) as json_file_open:
        params = json.load(json_file_open)
        try:
          s = params[param]
          sampler_counter[s] += 1.0
        except KeyError as e2:
          pass
    except Exception as e:
          print(e)
          time.sleep(10)

  return sampler_counter


def line_counter_from_path(path):
  # get counts of the line
  print(f"getting line counts from {path}")
  files = [str(f)
               for f in map(Path, sorted(glob.glob(f"{path}/*")))
               if f.is_file() and f.suffix in [".jpg", ".png"]
               ]
  print(f"got {len(files)} from {path}")

  line_counter = defaultdict(int)

  for f in files:
    # get the json file of the image
    file_path = Path(f)
    core_file_str = str(file_path.stem)
    timestamp = core_file_str.split("_")[0]
    json_file = file_path.parent / f"{timestamp}_settings.json"
    #print(f"json: {json_file}")
    try:
      with open(json_file) as json_file_open:
        params = json.load(json_file_open)
        # print(params)
        if 'line_number' in params:
          s = params['line_number'] # for here, just one
          line_counter[s] += 1.0
        else:
          line_counter["_"] += 1.0
          continue
    except Exception as e:
      print(f"line_counter_path {json_file}\n{e}")
      time.sleep(10)

  return line_counter

# get the line counts
print(f"\n\nLINES")

line_pos = {}
line_cnt = {}
neg_line = line_counter_from_path(deleted_path)
pos_line = line_counter_from_path(accepted_path)
skip_line = line_counter_from_path(skipped_path)
keep_line = line_counter_from_path(kept_path)

all_lines = set(list(neg_line.keys()) +
                list(pos_line.keys()) +
                list(skip_line.keys()) +
                list(keep_line.keys())
                )

# get the sorted proportionate samplers
print(f"all_lines: {all_lines} ")
sampler_pos = {}
for k in all_lines:

  try:
    vpos = pos_line[k] if k in pos_line else 0.0
    vneg = neg_line[k] if k in neg_line else 0.0
    spos = skip_line[k] if k in skip_line else 0.0
    kneg = keep_line[k] if k in keep_line else 0.0
    line_cnt[k] = vpos + vneg + spos + kneg
    # positive is accepted + skipped
    line_pos[k] = 1.0*(vpos + spos) / (vpos + vneg + spos + kneg)
  except:
    line_pos[k] = 1.0
    print(f"line_pos: no neg {k} ")

#print(sampler_pos)

sorted_line_pos = list((k,v) for k, v
                      in sorted(line_pos.items(),
                                key=lambda x: x[1], reverse=True)
)
print(f"sorted_line_pos: {sorted_line_pos}")
for k, v in sorted(sorted_line_pos,
                    key=lambda x: x[1], reverse=False):
  if type(k) == int:
    # have to ignore the unmatchable
    # print(f"k: {k}")
    # print(f"{line_pos[k]}")
    # print(f"{line_cnt[k]}")
    print(f"{k:2}: {line_pos[k]:4.3f} / {int(line_cnt[k]):4} :: {lines[k]}")
  else:
    print(f"{k:2}: {line_pos[k]:4.3f} / {int(line_cnt[k]):4} :: {k}")


##############################
print(f"\n\nPROMPTS")
for prompt_type in all_prompt_types:
  print(f"\n\nPROMPT: {prompt_type}")

  neg_counter, _, _ = prompt_counter_from_path(deleted_path, prompt_type)
  pos_counter, pos_rank, pos_count = prompt_counter_from_path(accepted_path, prompt_type)

  skip_counter, skip_rank, skip_count = prompt_counter_from_path(skipped_path, prompt_type)
  keep_counter, keep_rank, keep_count = prompt_counter_from_path(kept_path, prompt_type)

  # TODO: deal with integer, non-str prompts
  sorted_neg_prompts = list((k,v) for k, v in sorted(neg_counter.items(), key=lambda x: x[1], reverse=True))
  print(f"neg prompts: {sorted_neg_prompts}")
  neg_prompt_string = ", ".join([str(k) for k,v in sorted_neg_prompts[:15]])
  print(f"neg: {neg_prompt_string}")

  sorted_pos_prompts = list((k,v) for k, v in sorted(pos_counter.items(), key=lambda x: x[1], reverse=True))
  print(f"pos prompts: {sorted_pos_prompts}")
  pos_prompt_string = ", ".join([str(k) for k,v in sorted_pos_prompts[:15]])
  print(f"pos: {pos_prompt_string}")

  # get the highest and lowest proportionate prompts
  prop_pos = {}
  total_cnt = {}

  max_count = 0

  all_keys = set(list(neg_counter.keys()) +
                 list(pos_counter.keys()) +
                 list(skip_counter.keys()) +
                 list(keep_counter.keys())
                 )

  for k in all_keys:
    try:
      vneg = neg_counter[k] if k in neg_counter else 0.0
      vpos = pos_counter[k] if k in pos_counter else 0.0
      spos = skip_counter[k] if k in skip_counter else 0.0
      kneg = keep_counter[k] if k in keep_counter else 0.0
      total_cnt[k] = vpos + vneg + spos + kneg
      if total_cnt[k] > max_count:
        max_count =  total_cnt[k]

      # new 6/2023: use skips in numerator
      prop_pos[k] = 1.0*(vpos + spos) / total_cnt[k]
    except:
      prop_pos[k] = 1.0
      print(f"prop_pos: no neg {k} ")

  pt = prompt_type
  cur_prompt_gain = {}

  # make sure all the known ones at least get 1
  # so there's data at the beginning when not everything has been shown yet
  for k in eval(pt):
    cur_prompt_gain[k] = 1

  for k, v in sorted(prop_pos.items(), key=lambda x: (x[1], total_cnt[x[0]], x[0]), reverse=False):
    if k not in eval(pt):
      continue

    cur_prompt_gain[k] = 1 + int(prop_pos[k] * max_count)

    if total_cnt[k] >= min_count_to_show:
      print(f"{k:50}: gain={cur_prompt_gain[k]}, {prop_pos[k]:4.3f} / {int(total_cnt[k]):4}")

  # add to overall gain
  prompt_gain[pt] = cur_prompt_gain
  print(f"{pt}: total in prompt_gain: {len(prompt_gain[pt])}")

# save the prompt gain
gain_path = PROJECT_PATH / "prompt_gain.pkl"
import pickle

print(f"Saving gain to {gain_path}")
with open(gain_path, 'wb') as handle:
    pickle.dump(prompt_gain, handle, protocol=pickle.HIGHEST_PROTOCOL)





# Sequence exploration from accepted images (GPU required)


In [ ]:
#@title Extend a sequence from accepted images (adjust the order of the line numbers below)

#@markdown NOTE: you must open this cell and replace `lines_in_difficult_order` with the numbers in the `LINES` output from the final prompt votes in the cell above.


# Go through the accepted images
# See if the sequence folder already exists
# if not, create the folder and read the parameters
# go through each line

from pathlib import Path
import os, re, gc
import glob
import time
import json
from types import SimpleNamespace
from IPython import display

#@markdown if true, start from SeqAccepted
second_round_or_more = False #@param{type:"boolean"}

kill_when_done = False #@param{type:"boolean"}

#@markdown much faster if you don't show images
show_images = False #@param{type:"boolean"}


# move the accepted directories to the review path
if second_round_or_more:
  old_accepted_dirs = [f
                for f in map(Path, sorted(glob.glob(f"{sequence_accepted_path}/*")))
                if f.is_dir()
                ]
  print(f"Moving {len(old_accepted_dirs)} : {old_accepted_dirs}")

  # move these to the review path
  for d in old_accepted_dirs:
    print(f"moving: {d}")
    d.rename(sequence_review_path / d.name)

  # get the settings files from within those directories with an underscore
  # get the dirs
  accepted_dirs = [f
                for f in map(Path, sorted(glob.glob(f"{sequence_review_path}/*")))
                if f.is_dir()
                ]
  print(f"found {len(accepted_dirs)} review_paths")

  # get the most recent settings file from each dir
  accepted_files = []

  for d in accepted_dirs:
    settings_file = list(sorted(d.glob("*_settings.json")))[-1]
    print(f"adding settings {len(accepted_files)}: {settings_file}")
    accepted_files.append(settings_file)
  check_subdirs_only = True

else:
  # get the settings from the original accepted images
  accepted_files = [f
                for f in map(Path, sorted(glob.glob(f"{accepted_path}/*")))
                if f.is_file() and f.suffix in [".json"]
                ]
  check_subdirs_only = False

print(f"accepted files ({len(accepted_files)}): {accepted_files}")

# to minimize sequence testing
# take these from the final prompt votes

lines_in_difficult_order = [
 0,1


]

#@markdown how many lines to test, go: 2, 4, 8, 16, -1 (all)
max_lines_out = -1 #@param{type:"integer"}

if max_lines_out == -1:
  max_lines_out = len(lines_in_difficult_order)

lines_to_use = lines_in_difficult_order[:max_lines_out]
print(f"lines_to_use ({len(lines_to_use)}): {lines_to_use}")

max_accepted_files = -1 #@param{type:"integer"}

if max_accepted_files == -1:
  max_accepted_files = len(accepted_files)

accepted_files = accepted_files[:max_accepted_files]
print(f"accepted_files ({len(accepted_files)}): {accepted_files}")

# TODO: test different aspect ratios
# if set, the clear the line numbers and redo



def sequence_exists(sequence_dir):
  # check the possible paths

  in_review_path_already = False
  # TODO: need to double check this
  outdir = sequence_review_path / sequence_dir.name
  # print(f"Checking: {outdir}")
  if outdir.is_dir():
    in_review_path_already = True

  # TODO: need to double check this
  outdir = sequence_accepted_path / sequence_dir.name
  # print(f"Checking: {outdir}")
  if outdir.is_dir():
    if in_review_path_already:
      print(f"WARNING: accepted IN REVIEW: {outdir}")
    print(f"WARNING: sequence in accepted: {outdir}")
    return True

  # TODO: need to double check this
  outdir = sequence_deleted_path / sequence_dir.name
  # print(f"Checking: {outdir}")
  if outdir.is_dir():
    if in_review_path_already:
      print(f"WARNING: deleted IN REVIEW: {outdir}")
    print(f"WARNING: sequence in deleted: {outdir}")
    return True

  # TODO: outtakes are now just files
  # tODO: does this work?
  outdir = outtake_path / f"{sequence_dir.name.split('_')[0]}_settings.txt"
  # print(f"Checking: {outdir}")
  if outdir.is_file():
    if in_review_path_already:
      print(f"WARNING: outtake IN REVIEW: {outdir}")
    print(f"WARNING: sequence in outtake_path: {outdir}")
    return True

  return in_review_path_already

exceptions = []

for a in range(len(accepted_files)):
# for a in range(4):

  if second_round_or_more:
    # the directory is the parent of the file,

    settings_file = accepted_files[a]
    try:
      image_settings = get_settings_from_file(settings_file)
    except Exception as e:
      print(e)
      exceptions.append(e)
      continue

    outdir = accepted_files[a].parent
    cur_seed = image_settings.seed
    cur_timestamp = outdir.name.split("_")[0]
    cur_line_numbers = image_settings.line_numbers

  else:
    # the directory must be created in the for_review
    settings_file = accepted_files[a]
    image_settings = get_settings_from_file(settings_file)
    cur_seed = image_settings.seed
    cur_timestamp = settings_file.name.split("_")[0]

    # a new directory in review if starting from single images
    outdir = sequence_review_path / f"{cur_timestamp}_{cur_seed}"
    if sequence_exists(sequence_dir=outdir):
      print (f"EXISTS: {outdir}")
      continue
    cur_line_numbers = []

  print(f"\n\n({a:4}/{len(accepted_files):4}) :: dir: {outdir}")
  print(f"settings_file: {settings_file}")

  print(f"time: {cur_timestamp}")
  print(f"seed: {cur_seed}")

  print(f"cur_line_numbers: {cur_line_numbers}")

  # get an actual dict from the sn
  temp_dict = json.loads(json.dumps(image_settings, default=lambda s: vars(s)))

  # sets the global used by get_prompt_for_line below
  cur_core_prompt_dict = temp_dict['core_prompt_dict']

  # and turn it back
  image_settings.core_prompt_dict = cur_core_prompt_dict

  line_numbers = cur_line_numbers
  new_line_numbers = []

  # so the images show up in order, even if lines skipped
  for line_number in range(len(lines)):

    if line_number not in lines_to_use:
      continue
    if line_number in cur_line_numbers:
      # print(f"already run: {line_number}")
      continue
    new_line_numbers.append(line_number)
    line_numbers.append(line_number)

  if not new_line_numbers:
    print(f"already extended: {settings_file}")
    continue

  print(f"round: {a}; outdir: {outdir}; ")
  print(f"new_line_numbers: {new_line_numbers}")


  # need to make just one new settings file for all the images
  # so add prompts, image_files (plural) to the settings

  image_settings.line_numbers = line_numbers # all of them

  # uses cur_core_prompt_dict to create the actual prompt
  image_settings.prompts = [get_prompt_for_line(n) for n in line_numbers]

  print(f"scale: {image_settings.guidance_scale}; steps:{image_settings.steps}")
  print(f"final image_settings: {image_settings}")

  # save one setting for the whole directory
  save_settings(image_settings, settings_file.name, outdir)

  # go throught the prompts and files
  cur_image_settings = image_settings
  for n, line_num in enumerate(line_numbers):
    if line_num not in new_line_numbers:
      continue

    print(f"line {line_num}: ")
    # commment out for testing
    cur_image_settings.prompt = image_settings.prompts[n]
    print(f"cur_image_setting: {cur_image_settings}")
    output_image = get_one_image(cur_image_settings)

    if show_images:
      display.display(output_image)
    save_image_with_name(output_image, f"line_{line_num:02}", outdir)

  # clean up unused memory
  gc.collect()
  torch.cuda.empty_cache()

print(f"Exceptions: {exceptions}")

# kill switch when done
if kill_when_done:
  print("DONE!  Killing runtime")
  from google.colab import runtime
  runtime.unassign()




#**Sequence Voting (no GPU required)**




In [ ]:
#@title sequence voting functions

# show them to the user, if not in the file
# set A, S, D (accept, skip, delete) assessments
from IPython.core.interactiveshell import StrDispatch

from pathlib import Path
import glob
import time
import os
import json
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython import display
from collections import defaultdict
from PIL import Image, ImageFont, ImageDraw

def get_lines_from_path(path):
  # lines_to_use = [14, 2, 15, 11]
  settings_file = list(sorted(path.glob("*_settings.json")))[-1]
  print(f"settings_file: {settings_file}")
  settings = get_settings_from_file(settings_file)

  # lines_to_use = settings.line_numbers
  # image files are more informative
  lines_to_use = settings.prompts
  line_numbers = settings.line_numbers

  # the images are shown in original line order
  # so we need to reorder them

  ordered_lines = sorted(line_numbers)
  out_titles = []
  for n in ordered_lines:
    pos = line_numbers.index(n)
    out_titles.append(lines[pos])

  print(f"get_image_titles_from_path ({len(out_titles)}): {out_titles}")
  return out_titles


def image_grid_list(file_list, titles, max_height=960, title_replacements=[]):

  row2imgs = defaultdict(list)

  max_rows = 1
  # shoot for 3 rows max
  if len(file_list) <= 4:
    max_cols = len(file_list)
  else:
    max_cols = 4 if len(file_list) <= 12 else len(file_list) // 4
  max_w = 0
  max_h = 0


  for fnum, f in enumerate(file_list):
    img = Image.open(f)
    print(f"plot_images on row {max_rows}: fnum {fnum} file: {f}")

    max_w = max(max_w, img.size[0])
    max_h = max(max_h, img.size[1])

    row2imgs[max_rows].append(img)
    if fnum > 0 and (fnum+1) % max_cols == 0:
      max_rows += 1

  if len(row2imgs[max_rows]) == 0:
    max_rows -= 1

  font_size = 35
  max_title_length = int(max_w / int(font_size / 1.5))

  # print(f"row2imgs ({max_rows}, {max_cols}): {row2imgs}")
  print(f"row2imgs ({max_rows}, {max_cols})")
  print(f"max_w: {max_w}, max_h: {max_h}")

  grid = Image.new('RGB', size=(max_cols*max_w, max_rows*max_h))
  grid_w, grid_h = grid.size
  print(f"grid_size: {grid.size}")

  # use this if it the other cannot be found
  # font = ImageFont.load_default()
  font = ImageFont.truetype(r'/usr/share/fonts/truetype/liberation/LiberationMono-Regular.ttf', font_size)

  # add the line and position. inupper left
  draw = ImageDraw.Draw(grid)

  if len(file_list) == len(titles):
    add_title = True
  else:
    add_title = False

  i = 0
  for row, r in enumerate(row2imgs.keys()):
    for col, img in enumerate(row2imgs[r]):
      # print(f"{row}, {col} pasting at: {row*max_w} , {col*max_h}")
      grid.paste(img, box=(col*max_w, row*max_h))
      cur_title = ""
      if add_title:
        if title_replacements:
          cur_title = titles[i]
          for r in title_replacements:
            cur_title = cur_title.replace(r,'')
            cur_title = cur_title.strip()[:max_title_length]
        else:
          cur_title = titles[i][:max_title_length]
      i += 1

      draw.text((col*max_w +5, row*max_h+5), f"{cur_title}", fill='white', font=font)

  if grid_h > max_height:
    # shrink down so all the images can be seen at once
    ratio = max_height / grid_h
    new_h = max_height
    new_w = int(ratio * grid_w)
    print(f"resizing to ({new_w}, {new_h})")
    grid = grid.resize((new_w, new_h), Image.Resampling.LANCZOS)

  return grid


def remove_empty_dir(dir):
  #remove directory if empty
  if len(os.listdir(dir)) == 0:
    try:
      print(f"trying to remove empty dir: {dir}")
      os.rmdir(dir)
      print(f"\tremoved empty dir: {dir}")
    except Exception as e:
      print(e)
      pass

def move_dir_files_to_path(in_path, out_path):
  input_files = [f
               for f in map(Path, sorted(glob.glob(f"{in_path}/*")))
               if f.is_file()
               ]

  print(f"moving {len(input_files)} to {out_path}")
  # move the skipped files
  for file_path in input_files:
    try:
      file_path.rename(out_path / file_path.name)
      #print(f"move {f} TO {outtake_path / file_path.name}")
    except:
      print(f"ERROR moving: {f}")
      exit()

  remove_empty_dir(in_path)


In [ ]:
#@title Sequence Voting v2


###########################
review_dirs = [
          f for f in map(Path, sorted(glob.glob(f"{sequence_review_path}/*")))
               if f.is_dir()  and "_" in str(f)
    ]

print(f"review_dirs ({len(review_dirs)}): {review_dirs}")

quit = False

total_dirs = len(review_dirs)

accept_cnt = 0
skip_cnt = 0
delete_cnt = 0

# if there's some repetition among the lines
title_replacements = []

for i, sd in enumerate(review_dirs):
  if quit:
    break

  if i % 10 == 0:
    display.clear_output(wait=True)

  print(f"\n\n ({i+1:2} / {total_dirs:3}) {accept_cnt:3}/{skip_cnt:3}/{delete_cnt:3}\n{sd.name}")
  # lines_to_use = get_lines_from_path(sd)
  # titles_to_use = [lines[i][0] if type(lines[i]) == list else lines[i]  for i in lines_to_use  ]
  # titles_to_use = lines_to_use

  files = [
      f
      for f in map(Path, sorted(glob.glob(f"{sd}/*")))
      if f.is_file() and f.suffix in [".jpg", ".png"]
    ]

  titles_to_use = []
  for f in files:
    line_num = int(f.stem.split("_")[1])
    titles_to_use.append(lines[line_num])

  grid = image_grid_list(files, titles=titles_to_use, title_replacements=title_replacements)
  display.display(grid)

  accepted = False
  time.sleep(6) # 8 needs at

  # have to have this sleep for the input box to show up
  while not accepted:
    try:
      someInput = input("Accept: ")
      new_path = None
      if someInput == "a":
        accepted = True
        new_path = sequence_accepted_path / sd.name
        accept_cnt += 1
      elif someInput == "d":
        accepted = True
        new_path = sequence_deleted_path / sd.name
        delete_cnt += 1
      elif someInput == "s":
        accepted = True
        # TODO: need to move these to Keepers and individual images
        new_path = outtake_path / sd.name
        skip_cnt += 1
      elif someInput == "q":
        print("Quitting")
        accepted = True
        quit = True
      if new_path:
        print(f"moving {sd}  to {new_path}")
        sd.rename(new_path)

    except Exception as e:
      print(e)





In [ ]:
#@title adjust image settings in sequences from skips to accepted (CPU)

# go through the skip sequences (folders in skips)
# decide if they can be recovered by adding to negative prompts
# if so, then add (from input) to the negative prompt list, remove the line numbers
# put the settings file in to SeqAccepted for the next round
import os
from datetime import datetime

#@markdown to restrict what to review: "%Y-%m-%d %H:%M:%S", e.g., 2024-01-26 12:00:00
start_time_text = "" #@param{"type":"string"}

if start_time_text:
  start_time = datetime.strptime(start_time_text,"%Y-%m-%d %H:%M:%S")
else:
  start_time = None

print(f"start_time: {start_time}")

###########################
# get skipped sequences
skip_dirs = [
          f for f in map(Path, sorted(glob.glob(f"{outtake_path}/*")))
               if f.is_dir()  # and "_" in str(f)
    ]

print(f"skip_dirs ({len(skip_dirs)}): {skip_dirs}")

quit = False

total_dirs = len(skip_dirs)

accept_cnt = 0

# if there's some repetition among the lines
title_replacements = []

for i, sd in enumerate(skip_dirs):
  if quit:
    break

  dir_time = datetime.fromtimestamp(os.path.getmtime(sd))
  print(f"dir_time: {dir_time}")
  if start_time and dir_time < start_time:
    print(f"APPROVED from {os.path.getctime(sd)}: {sd}")
    continue


  if i % 10 == 0:
    display.clear_output(wait=True)

  print(f"\n\n ({i+1:2} / {total_dirs:3}) {accept_cnt:3}\n{sd.name}")

  files = [
      f
      for f in map(Path, sorted(glob.glob(f"{sd}/*")))
      if f.is_file() and f.suffix in [".jpg", ".png"]
    ]

  titles_to_use = []
  for f in files:
    line_num = int(f.stem.split("_")[1])
    titles_to_use.append(lines[line_num])

  grid = image_grid_list(files, titles=titles_to_use, title_replacements=title_replacements)
  display.display(grid)

  accepted = False
  # have to have this sleep for the input box to show up
  time.sleep(6) # 8 needs at

  negative_terms = None
  while not accepted:

    try:
      someInput = input("new neg?: ")
      new_path = None
      if someInput == "s":
        print("Skipping")
        accepted = True
      elif someInput == "q":
        print("Skipping")
        accepted = True
        quit = True
      elif len(someInput) > 3:
        # get the negative terms to add
        negative_terms = someInput
        accepted = True

    except Exception as e:
      print(e)

  if negative_terms:
    settings_file = list(sorted(sd.glob("*_settings.json")))[-1]
    print(f"using settings: {settings_file}")
    image_settings = get_settings_from_file(settings_file)

    # get a new seed to keep track of the change
    image_settings.seed = random.randint(1,2147483647)
    cur_timestamp = settings_file.name.split("_")[0]

    # get an actual dict from the sn
    temp_dict = json.loads(json.dumps(image_settings, default=lambda s: vars(s)))

    # sets the global used by get_prompt_for_line below
    cur_core_prompt_dict = temp_dict['core_prompt_dict']

    # adjust the core promp_dict
    print(f"current negative_qualities: {cur_core_prompt_dict['negative_qualities']}")

    new_initial_terms = [t.strip() for t in negative_terms.split(",")]
    print(f"new_initial_terms: {new_initial_terms}")

    cur_core_prompt_dict['negative_qualities'] = new_initial_terms + cur_core_prompt_dict['negative_qualities']
    print(f"new negative_qualities: {cur_core_prompt_dict['negative_qualities']}")

    # and turn it back
    image_settings.core_prompt_dict = cur_core_prompt_dict

    image_settings.line_numbers = [] # empty the line numbers

    outdir = sequence_accepted_path / f"{cur_timestamp}_{image_settings.seed}"

    # save one setting for the whole directory
    save_settings(image_settings, settings_file.name, outdir)




# Step 3. Random seed testing (GPU required)

In [ ]:
#@title Given an accepted settings file, randomize the seeds for each line

# Go through the accepted images
# create 10 versions of each line with a different seed
# try to find a path through the images that is consistent
# if not, create the folder and read the parameters
# go through each line

from pathlib import Path
import os, re, gc
import glob
import time
import json
from types import SimpleNamespace
from IPython import display

rounds = 2 #@param{type:"integer"}

kill_when_done = False #@param{type:"boolean"}

#@markdown much faster if you don't show images
show_images = False #@param{type:"boolean"}

#@markdown within SeqAccepted if you want to do just one
settings_dir =  "" #@param{type:"string"}

#@markdown put a seed here to get this one as part of the set
original_seed = 0 #@param{type:"integer"}

if settings_dir:
  # do just 1
  settings_paths = [sequence_accepted_path / settings_dir]
else:
  # go through all the directories in sequence_accepted_path
  settings_paths = [
    f
    for f in map(Path, sorted(glob.glob(f"{sequence_accepted_path}/*")))
    if f.is_dir()
  ]

print(f"Settings paths ({len(settings_paths)}): {settings_paths}")

# if you don't want to run all of the lines adjust this
lines_to_use = list(range(0, len(lines)))
# or
# lines_to_use = [0, 1, 7, 9, 10, 11, 12, 13,14, 16, 18, 19, 25, 28, 31, 36, 39, 42, 49, 54, 57, ]

for settings_path in settings_paths:
  print(f"settings_path: {settings_path}")
  # get the last one
  settings_file = list(sorted(settings_path.glob("*_settings.json")))[-1]
  print(f"adding settings: {settings_file}")

  # get the parent's timestamp
  # make this directory to hold the seed directories
  settings_timestamp = settings_path.name.split("_")[0]
  print(f"settings_timestamp: {settings_timestamp}")

  # settings timestamp tells us where the original settings came from
  timestamp_dir = sequence_review_path / settings_timestamp
  os.makedirs(timestamp_dir, exist_ok=True)
  print(f"saving seed directories under: {timestamp_dir}")

  for round in range(rounds):
    # to prevent errors
    if round+1 % 3==0:
      display.clear_output(wait=True)

    if original_seed and round == 0:
      cur_seed = original_seed
    else:
      # new random seed
      cur_seed = random.randint(1,4294967295) # numpy limit

    # new settings format
    image_settings = get_settings_from_file(settings_file)
    # all the individual lines will get this same new random seed
    image_settings.seed = cur_seed

    # get an actual dict from the simplenamespace
    temp_dict = json.loads(json.dumps(image_settings, default=lambda s: vars(s)))
    cur_core_prompt_dict = temp_dict['core_prompt_dict']
    # and turn it back
    image_settings.core_prompt_dict = cur_core_prompt_dict

    # so each round will have a different cur_seed and directory under timestamp_dir
    outdir_name = f"{settings_timestamp}_{cur_seed}"
    outdir = timestamp_dir / outdir_name
    os.makedirs(outdir, exist_ok=True)

    print(f"\nRound {round}: outdir: {outdir}; seed: {cur_seed}")

    # need to make just one new settings file for all the images
    # so add prompts, image_files to the settings

    line_numbers = list(range(0, len(lines))) # get all the lines
    image_settings.line_numbers = line_numbers
    image_settings.prompts = [get_prompt_for_line(n) for n in line_numbers]

    print(f"guidance_scale: {image_settings.guidance_scale}; steps:{image_settings.steps};")
    print(f"final image_settings: {image_settings}")

    # save one setting for the whole directory;
    # have to define the filename completely
    setting_filename = f"{outdir_name}_settings.json"
    save_settings(image_settings, setting_filename, outdir)

    # go through the prompts and files
    cur_image_settings = image_settings
    for line_num in line_numbers:
      if line_num not in lines_to_use:
        continue

      print(f"creating image for line {line_num}: '{image_settings.prompts[line_num]}'")
      # commment out for testing:
      cur_image_settings.prompt = image_settings.prompts[line_num]
      print(f"cur_image_setting: {cur_image_settings}")

      # if run_mode == "Interpolation":
      #   # to match what happens with interpolation base_fulle
      #   output_image = get_one_image_from_base_full(cur_image_settings)
      # else:
      output_image = get_one_image(cur_image_settings)

      if show_images:
        display.display(output_image)
      save_image_with_name(output_image, f"line_{line_num:02}", outdir)

    # clean up unused memory
    gc.collect()
    torch.cuda.empty_cache()

# kill switch when done
if kill_when_done:
  print("DONE!  Killing runtime")
  from google.colab import runtime
  runtime.unassign()



In [ ]:
#@title (if necc, do once) copy all accepted dirs into the SeqForReview subfolders (no GPU)
from pathlib import Path
import glob
from distutils.dir_util import copy_tree

settings_paths = [
    f
    for f in map(Path, sorted(glob.glob(f"{sequence_accepted_path}/*")))
    if f.is_dir()
  ]

print(f"Found {len(settings_paths)} setting_paths: {settings_paths}")

for settings_path in settings_paths:
  settings_timestamp = settings_path.name.split("_")[0]
  print(f"settings_timestamp: {settings_timestamp}")

  # settings timestamp tells us where the original settings came from
  timestamp_dir = sequence_review_path / settings_timestamp
  # save a copy in the review path
  new_path = timestamp_dir / settings_path.name

  copy_tree(str(settings_path), str(new_path))

  # settings_path.copy(new_path)
  print(f"copied {settings_path} TO {new_path}")
  # print(f"will copy {settings_path} TO {new_path}")




In [ ]:
#@title Choose the best images for each set (no GPU)

# Go through the accepted images
# create 10 versions of each line with a different seed
# try to find a path through the images that is consistent
# if not, create the folder and read the parameters
# go through each line

from pathlib import Path
import os, re, gc
import glob
import time
import json
from types import SimpleNamespace
from IPython import display
from collections import defaultdict

import cv2
from matplotlib import pyplot as plt
import imageio

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from PIL import Image, ImageFont, ImageDraw

import shutil

#@markdown otherwise copy chosen images to SeqFinal/{timestamp}
just_output_seeds = False #@param{type:"boolean"}

#@markdown otherwise do all dirs in SeqForReview
one_timestamp = "" #@param{type:"string"}



###################

def image_grid_dict(file_dict, titles, max_height=720):
  # images = []

  row2imgs = defaultdict(list)

  max_rows = len(file_dict.keys())
  max_cols = 1
  max_w = 0
  max_h = 0
  # Create a font object
  # font = ImageFont.truetype('arial.ttf', 32)

  for r in sorted(file_dict.keys()):
    for fnum, f in enumerate(file_dict[r]):
      # print(f"plot_images: file: {f}")
      # display.display(Image.open(f))
      # img = cv2.cvtColor(cv2.imread(str(f)), cv2.COLOR_BGR2RGB)
      img = Image.open(f)
      # print(f"plot_images {r}: file: {f}")

      max_w = max(max_w, img.size[0])
      max_h = max(max_h, img.size[1])

      # add the line and position. inupper left
      # draw = ImageDraw.Draw(img)
      # draw.text((15,15), f"{r}:{fnum}", fill='white', size=400)

      row2imgs[r].append(img)
      if len(row2imgs[r]) > max_cols:
        max_cols = len(row2imgs[r])

  # print(f"row2imgs ({max_rows}, {max_cols}): {row2imgs}")
  print(f"row2imgs ({max_rows}, {max_cols})")
  print(f"max_w: {max_w}, max_h: {max_h}")


  grid = Image.new('RGB', size=(max_cols*max_w, max_rows*max_h))
  grid_w, grid_h = grid.size
  print(f"grid_size: {grid.size}")

  # font = ImageFont.load_default()
  font = ImageFont.truetype(r'/usr/share/fonts/truetype/liberation/LiberationMono-Regular.ttf', 70)

  # add the line and position. inupper left
  draw = ImageDraw.Draw(grid)
  # draw.text((15,15), f"{r}:{fnum}", fill='white', size=400)

  for row, r in enumerate(row2imgs.keys()):
    for col, img in enumerate(row2imgs[r]):
      # print(f"{row}, {col} pasting at: {row*max_w} , {col*max_h}")
      grid.paste(img, box=(col*max_w, row*max_h))
      draw.text((col*max_w +5, row*max_h+5), f"{r}:{col}", fill='white', font=font)

  # for i, img in enumerate(images):
  #     grid.paste(img, box=(i%max_cols*max_w, i//cols*max_h))

  if grid_h > max_height:
    # shrink down so all the images can be seen at once
    ratio = max_height / grid_h
    new_h = max_height
    new_w = int(ratio * grid_w)
    print(f"resizing to ({new_w}, {new_h})")
    grid = grid.resize((new_w, new_h), Image.Resampling.LANCZOS)

  return grid




#############################################
# 1. get all the unique set names
print(f"sequence_review_path: {sequence_review_path}")

if one_timestamp:
  initial_timestamps = [str(sequence_review_path/ one_timestamp)]
else:
  # go through all the directories in the review path
  initial_timestamps = list(sorted(set(
    f.name.split("_")[0]
    for f in map(Path, sorted(glob.glob(f"{sequence_review_path}/*")))
    if f.is_dir()))
  )

print(f"initial_timestamps {len(initial_timestamps)}: {initial_timestamps}")

# how much to show at a time.
rows_at_a_time = 3

for ts_num, init_ts in enumerate(initial_timestamps):

  # 2. get the whole set of images for each line number
  print()
  print(f"{ts_num:02} cur_set: {init_ts}")

  # go through the directories that start with that
  cur_versions = [f
    for f in map(Path, sorted(glob.glob(f"{sequence_review_path}/{init_ts}/*")))
    if f.is_dir()]

  print(f"\tcur_versions ({len(cur_versions)}): {cur_versions}")

  line2files  = defaultdict(list)

  # keep track of the version/seed that was shown and chosen
  line2seed = defaultdict(list)

  image_settings = None

  for version_num, version in enumerate(cur_versions):
    # get the images for each line number,
    # check if they are all there
    print(f"\t{ts_num:02} {version_num:02} Analyzing: {version}")

    # get the settings file
    if not image_settings:
      try:
        # some older versions don't have a proper settings file
        cur_settings_file = list(sorted(version.glob(f"*_settings.json")))[-1]
        # cur_settings_file = sorted(version.glob(f"*_settings.json"))
        print(f"{cur_settings_file}")
        image_settings = get_settings_from_file(cur_settings_file)
      except:
        print(f"missing settings for: {version}")

    cur_seed = version.stem.split("_")[-1]


    # need to keep track of the seed that made each chosen image
    seeds_all = []

    # get the lines in this directory
    line_files = [
      f
      for f in map(Path, sorted(glob.glob(f"{version}/*")))
      if f.is_file()  and f.suffix in [".jpg", ".png"]
      ]

    print(f"line_files: {len(line_files)}: {line_files}")

    # set the line to file dict
    for line_num, lf in enumerate(line_files):
      lf_str = lf.stem
      lf_parts = lf_str.split("_")
      # print(f"line_parts: {lf_parts}")
      line = int(lf_parts[1])
      # print(f"{ts_num:02} {version_num:02} {line_num}: {line:02}: {lf}")
      line2files[line].append(lf)
      line2seed[line].append(cur_seed)

  print()
  total_keys_cnt = len(line2files.keys())
  print(f"total keys: {total_keys_cnt}: line2files: {line2files}")

  print(f"line2seed:  {line2seed}")

  if total_keys_cnt != len(lines):
    # if just looking at a few and outputting seeds, this is okay
    print(f"WARNING: Not examples for each line: for {init_ts}")
    each_line = set(range(len(lines)))
    cur_lines = set(line2files.keys())
    print(f"missing lines: {each_line - cur_lines}")


  # need to just approve 3 at a time
  cur_start = 0

  choices_all = []
  accepted_all = False

  line2chosen_seed = {}

  while cur_start < len(lines):
    print(f"line2files: {line2files}")
    cur_file_dict = defaultdict(list)
    for line_num in range(cur_start, min(len(lines), cur_start+rows_at_a_time)):
      if not line2files[line_num]:
        # if we skipped some
        continue
      cur_file_dict[line_num].extend(line2files[line_num])

    if len(cur_file_dict) == 0:
      cur_start += rows_at_a_time
      continue

    print(f"cur_file_dict: {cur_file_dict}")

    image_grid_dict(cur_file_dict, titles=None)

    for ln in cur_file_dict.keys():
      print(f"line ({int(ln)}): {lines[int(ln)]}")

    accepted = False

    time.sleep(8) # 8 needs at


    # have to have this sleep for the input box to show up
    while not accepted:
      try:
        someInput = input("Accept: ")
        new_path = None
        if re.match("^[0-9 ]+$", someInput):
          # just numbers and spaces
          someInput = someInput.strip()
          choices = someInput.split()
          if len(choices) == len(cur_file_dict):
            # we have a choice for each line
            accepted = True
            print(f"choices: {choices}")
            choices_all.extend([int(c) for c in choices])
            # get the seed for each choice
            seeds_all.extend([int(line2seed[line_num][int(c)]) for line_num, c in zip(cur_file_dict.keys(), choices)])
            for line_num, c in zip(cur_file_dict.keys(), choices):
              line2chosen_seed[line_num] = int(line2seed[line_num][int(c)])
            image_settings.seeds_all = seeds_all
            print(f"seeds_all: {seeds_all}")
            accepted_all = True
          else:
            print(f"Not enough choices: {len(choices)}")
            accepted = False
        elif someInput == "q":
          print("Quitting")
          accepted = True
          quit = True
        elif someInput == "s":
          print("Skipping")
          accepted = True

      except Exception as e:
        print(e)
        pass

    cur_start += rows_at_a_time


  for line_num in line2chosen_seed.keys():
    print(f"line2seed: {line_num}: {line2chosen_seed[line_num]}")

  if choices_all and accepted_all:
    if just_output_seeds:
      print(f"seeds_all: {image_settings.seeds_all}")
    else:

      print(f"init_ts: {type(init_ts)}: {init_ts}")
      init_ts_dir = init_ts.split("/")[-1] # this is a string
      move_to_dir = sequence_final_path / init_ts_dir
      print(f"Saving final files to: {move_to_dir}")

      os.makedirs(move_to_dir, exist_ok=True)
      setting_filename = f"{init_ts_dir}_settings.json"
      save_settings(image_settings, setting_filename, move_to_dir)

      # get the actual file from choices
      for line_num, line_key in enumerate(sorted(line2files.keys())):
        print(f"choices: {line_num}: {line_key}: {line2files[line_key]}")
        chosen_file = line2files[line_key][int(choices_all[line_num])]
        new_name = "_".join(chosen_file.name.split("_")[1:]) # get rid of the timestamp
        new_path = move_to_dir / new_name
        print(f"choices: copying\n\t{chosen_file}\n\t{new_path}")
        shutil.copy(chosen_file, new_path)



